# Calculate Spatial and temporal NME scores, then calculate ensemble weights.

In [1]:
import os

import numpy as np
import pandas as pd
import warnings

from Functions import *
from Global_Variables import *

In [2]:
obs_df = pd.read_pickle(os.path.join(RESULTS_PATH, 'AR6_obs_df.pkl'))
obsclim_df = pd.read_pickle(os.path.join(RESULTS_PATH, 'AR6_obsclim_df.pkl'))
counterclim_df = pd.read_pickle(os.path.join(RESULTS_PATH, 'AR6_counterclim_df.pkl'))

## Calculate NME scores

### Spatial NME

In [3]:
warnings.filterwarnings('ignore') # Surpress some unimportant warnings
rng = np.random.default_rng(SEED)
start_year, end_year = 2003, 2019
NME_scores = pd.DataFrame(index=pd.MultiIndex.from_product([obs_dict.keys(), ('$NME_1$', '$NME_3$')], names=['Observation', 'NME']), columns=obsclim_df.columns.unique(level='Model'))

for obsname in NME_scores.index.unique(level='Observation'):
    obs = constrain_time(iris.load_cube(obs_dict[obsname]), start_year, end_year)
    for modelname in NME_scores.columns.unique(level='Model'):
        model = constrain_time(preprocess_model(iris.load_cube(model_dict[modelname]['obsclim']), modelname), start_year, end_year)
        NME_scores.loc[(obsname), modelname] = [NME1(obs, model), NME3(obs, model)]

warnings.filterwarnings('default')   
NME_scores.to_pickle(os.path.join(RESULTS_PATH, 'NME_scores_spatial.pkl'))       
NME_scores

Model                 CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE  \
Observation NME                                                   
FireCCI5.1  $NME_1$  0.892374   0.78234                0.781677   
            $NME_3$  0.738435  0.768493                0.791827   
GFED5       $NME_1$  0.713656   0.69347                0.721336   
            $NME_3$  0.770402  0.768219                0.826412   

Model               LPJ-GUESS-SPITFIRE ORCHIDEE-MICT-SPITFIRE     SSiB4  \
Observation NME                                                           
FireCCI5.1  $NME_1$           0.844539               0.895032  0.648045   
            $NME_3$           0.820376               0.802746  0.648416   
GFED5       $NME_1$           0.773655               0.734205  0.622984   
            $NME_3$           0.880054               0.816251  0.727108   

Model                   VISIT  
Observation NME                
FireCCI5.1  $NME_1$  1.165131  
            $NME_3$  1.060099  
GFED5       $NME_1$    1.0263  
            $NME_3$    1.1412

### Temporal NME

#### Global

In [4]:
index = pd.MultiIndex.from_product([obs_dict.keys(), ('NME3_ranked', 'NME3_annual')], names=['Observation', 'NME'])
NME_scores_global = pd.DataFrame(index=index, columns=obsclim_df.columns.unique(level='Model'))
start_year, end_year = 2003, 2019

for obsname, nmename in NME_scores_global.index:
    obs = to_global(constrain_time(select_models(obs_df, obsname), start_year, end_year))
    for modelname in NME_scores_global.columns:
        model = to_global(constrain_time(select_models(obsclim_df, modelname), start_year, end_year))
        NME_scores_global.loc[(obsname), modelname] = (NME3_temporal(obs.sort_values(by=obsname)[obsname], model.sort_values(by=modelname)[modelname]), NME3_temporal(to_annual(obs)[obsname], to_annual(model)[modelname]))
        
NME_scores_global.to_pickle(os.path.join(RESULTS_PATH, 'NME_scores_global_temporal.pkl'))
NME_scores_global

Model                     CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE  \
Observation NME                                                       
FireCCI5.1  NME3_ranked  0.846836  0.922562                0.801384   
            NME3_annual   0.70586  0.845718                1.081127   
GFED5       NME3_ranked  0.868052  0.935159                0.910219   
            NME3_annual   0.73154  0.923797                0.940393   

Model                   LPJ-GUESS-SPITFIRE ORCHIDEE-MICT-SPITFIRE     SSiB4  \
Observation NME                                                               
FireCCI5.1  NME3_ranked           0.944959               1.410181  0.536967   
            NME3_annual           1.256442               1.664964  0.781577   
GFED5       NME3_ranked           1.167431               1.487771   0.73239   
            NME3_annual           1.395473               1.829899  0.727651   

Model                       VISIT  
Observation NME                    
FireCCI5.1  NME3_ranked  1.210021  
            NME3_annual  1.576751  
GFED5       NME3_ranked  1.263276  
            NME3_annual  1.516288

#### Regional

Calculate __for__ each region, __for__ each of the the two observations the temporal NME3 scores (annual and ranked) of the _fire models_ and the _reference model_ (randomly resampled observations).

In [ ]:
rng = np.random.default_rng(SEED)
start_year, end_year = 2003, 2019
index = pd.MultiIndex.from_product([obs_dict.keys(), ('NME3_ranked', 'NME3_annual')], names=['Observation', 'NME'])
NME_scores = pd.DataFrame(index=index, columns=obsclim_df.columns)

columns = obsclim_df.columns.unique(level='Region')
NME_ref_scores = pd.DataFrame(index=index, columns=columns)

for regionname, modelname in NME_scores.columns:
    obs = constrain_time(select_region(obs_df, regionname), start_year, end_year)
    model = constrain_time(select_region(select_models(obsclim_df, modelname), regionname), start_year, end_year)
       
    for obs_name in NME_scores.index.unique(level='Observation'):
        obs_select = select_models(obs, obs_name)
        obs_series, model_series = obs_select[(regionname, obs_name)], model[(regionname, modelname)]
        obs_annual_series, model_annual_series = to_annual(obs_select)[(regionname, obs_name)], to_annual(model)[(regionname, modelname)]
        
        zero_obs = pd.Series(0, index=obs_select.index)
        mean_obs = obs_select.mean(axis=1)
        random_obs = pd.Series(rng.choice(obs_select.values.flatten(), size=len(obs.index), replace=False), index=obs_select.index)
        
        NME_scores.loc[(obs_name), (regionname, modelname)] = (NME3_temporal(obs_series.sort_values(), model_series.sort_values()), NME3_temporal(obs_annual_series, model_annual_series))
        NME_ref_scores.loc[(obs_name), (regionname)] = [NME3_temporal(obs_series.sort_values(), random_obs), NME3_temporal(to_annual(obs_series), to_annual(random_obs))]

NME_scores.to_pickle(os.path.join(RESULTS_PATH, 'NME_scores_temporal.pkl'))
NME_ref_scores.to_pickle(os.path.join(RESULTS_PATH, 'NME_ref_scores_temporal.pkl'))
NME_scores

Region                        NWN                                    \
Model                     CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE   
Observation NME                                                       
FireCCI5.1  NME3_ranked  0.667634   0.59398                0.560215   
            NME3_annual  1.581303   1.20321                 1.34238   
GFED5       NME3_ranked  0.770548   0.60962                0.617835   
            NME3_annual  1.652451  1.289971                 1.31332   

Region                                                                       \
Model                   LPJ-GUESS-SPITFIRE ORCHIDEE-MICT-SPITFIRE     SSiB4   
Observation NME                                                               
FireCCI5.1  NME3_ranked           0.523431               0.671359  0.803311   
            NME3_annual           0.931624               1.703716  0.834731   
GFED5       NME3_ranked           0.509879               0.780018  0.730218   
            NME3_annual           0.981308               1.732666  0.925459   

Region                                  NEN                                    \
Model                       VISIT   CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE   
Observation NME                                                                 
FireCCI5.1  NME3_ranked  1.383773  0.800999  0.612783                0.702568   
            NME3_annual  1.337219   1.19589  1.198015                1.660582   
GFED5       NME3_ranked  1.455793  1.081188   0.88838                1.008184   
            NME3_annual  1.368974  1.168029  1.035364                1.634643   

Region                   ...                    SAU                      \
Model                    ... ORCHIDEE-MICT-SPITFIRE     SSiB4     VISIT   
Observation NME          ...                                              
FireCCI5.1  NME3_ranked  ...               1.036854  1.162261  1.283582   
            NME3_annual  ...               1.477326  1.227452  1.183228   
GFED5       NME3_ranked  ...               1.300672  1.401575  1.291352   
            NME3_annual  ...               1.547022   0.95716  1.419682   

Region                         NZ                                    \
Model                     CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE   
Observation NME                                                       
FireCCI5.1  NME3_ranked  0.986952  1.178521                1.297308   
            NME3_annual  1.062708  1.071935                1.363532   
GFED5       NME3_ranked  1.371506  1.315251                1.527835   
            NME3_annual  1.048551   0.82221                1.499665   

Region                                                                       \
Model                   LPJ-GUESS-SPITFIRE ORCHIDEE-MICT-SPITFIRE     SSiB4   
Observation NME                                                               
FireCCI5.1  NME3_ranked           1.127771               1.011746  1.169803   
            NME3_annual           1.196678               1.070684  1.388105   
GFED5       NME3_ranked           1.193844               1.284824  1.305172   
            NME3_annual            1.02949               0.996795  1.085325   

Region                             
Model                       VISIT  
Observation NME                    
FireCCI5.1  NME3_ranked  1.464417  
            NME3_annual  1.217055  
GFED5       NME3_ranked   1.39587  
            NME3_annual  1.302082  

[4 rows x 301 columns]

## Calculate Weights

#### Global

In [6]:
sigmaD = 0.5
NME_scores = pd.read_pickle(os.path.join(RESULTS_PATH, 'NME_scores_global_temporal.pkl'))

columns = pd.MultiIndex.from_tuples(list(map(lambda x: ('Global', x), NME_scores.columns)), names=['Region', 'Model'])
model_weights_global = pd.DataFrame(np.exp(-NME_scores.values.astype(float)/sigmaD).sum(axis=0)[None, :], index=['weights'], columns=columns)
model_weights_global /= model_weights_global.sum(axis=1).values[0]
model_weights_global

Region     Global                                                       \
Model     CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE LPJ-GUESS-SPITFIRE   
weights  0.211047  0.165229                0.159386           0.098615   

Region                                              
Model   ORCHIDEE-MICT-SPITFIRE     SSiB4     VISIT  
weights               0.043493  0.256602  0.065628

#### Regional

In [7]:
sigmaD = 0.5
NME_scores = pd.read_pickle(os.path.join(RESULTS_PATH, 'NME_scores_temporal.pkl'))

model_weights = pd.DataFrame(np.exp(-NME_scores.values.astype(float)/sigmaD).sum(axis=0)[None, :], index=['weights'], columns=NME_scores.columns)
model_weights /= model_weights.groupby(level='Region', axis=1).sum()
model_weights

Region        NWN                                                       \
Model     CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE LPJ-GUESS-SPITFIRE   
weights  0.119577  0.164712                0.162804           0.216555   

Region                                                   NEN            \
Model   ORCHIDEE-MICT-SPITFIRE     SSiB4     VISIT   CLASSIC     JULES   
weights               0.115149  0.167276  0.053926  0.149697  0.201669   

Region                           ...                    SAU            \
Model   LPJ-GUESS-SIMFIRE-BLAZE  ... ORCHIDEE-MICT-SPITFIRE     SSiB4   
weights                0.134243  ...               0.118875  0.156642   

Region                  NZ                                    \
Model      VISIT   CLASSIC     JULES LPJ-GUESS-SIMFIRE-BLAZE   
weights  0.12179  0.167461  0.179333                0.089089   

Region                                                                 
Model   LPJ-GUESS-SPITFIRE ORCHIDEE-MICT-SPITFIRE     SSiB4     VISIT  
weights           0.156214                0.17384  0.130158  0.103906  

[1 rows x 301 columns]

#### Merge the two and save

In [8]:
pd.concat([model_weights, model_weights_global], axis=1).to_pickle(os.path.join(RESULTS_PATH, 'NME3_Weights.pkl'))